In [ ]:
%load_ext autoreload
%autoreload 2
from Methods import sentenceSimilarityMethods,JobOffers,dataframeHelpers

In [2]:
import json
import os
import nltk
import re
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

In [3]:
from scipy import spatial

In [4]:
model = hub.load("/Users/klong/Desktop/Thesis/universal-sentence-encoder_4/")
def embed(input):
  return model(input)

In [5]:
autoreload JobOffers,dataframeHelpers

In [ ]:
# Only run this if you want to redo the translations
#JobOffers.writeTranslatedJobs(100)

In [6]:
slimmedActsDf = dataframeHelpers.buildActivitiesDFV26()

In [7]:
slimmedActsDf

,ACTIVITY_CONTENT,ACTIVITY_ID
0,Where and how are the information and data gai...,2
1,"Observing, receiving, and otherwise obtaining...",3
2,Research geological features or processes.,4
3,Conduct climatological research.,5
4,Investigate the environmental impact of indust...,6
...,...,...
283,Providing guidance and expert advice to manage...,290
284,Performing day-to-day administrative tasks suc...,291
285,Obtain permits.,292
286,"Recruiting, interviewing, selecting, hiring, ...",293


In [8]:
#got stuck here
jobsDf = dataframeHelpers.buildJobsDF("Data/output_V1.1.1000.translate.json")

In [9]:
#jobsDf.to_csv("1000TranslatedJobs.csv")

In [10]:
jobsDf["JOB_CONTENT"][0]

['We are a successful and independent engineering office in Schönried',
 ' To strengthen our team and to optimally process the manifold interesting objects, we are looking for civil engineering MW of their work areas of constructive engineering construction both in project planning and in the construction management of general low and road construction',
 ' Your profile You have a completed studies as a civil engineer ETHTUFH',
 ' You want to gain experience in configuration and construction management',
 ' They are all over and want to develop develops are used to deal with pressure and can also retain the tranquility in hectic situations',
 ' Your chances You can extend your knowledge and experience in a small motivated team with modern work equipment and interesting objects',
 ' You will get the opportunity to grow on the task',
 ' Further information and application We are at your disposal for more information and look forward to receiving your application',
 ' Please send your doc

In [11]:
embeddings = dataframeHelpers.getEmbeddings(slimmedActsDf, jobsDf)

In [12]:
slimmedActsDf[slimmedActsDf["ACTIVITY_CONTENT"].isna()]


,ACTIVITY_CONTENT,ACTIVITY_ID


In [13]:
jobsSentences = np.array([y for x in jobsDf["JOB_CONTENT"] for y in x])

In [14]:
jobsDf = dataframeHelpers.addEmbeddingsToJobs(jobsDf, embeddings)
slimmedActsDf = dataframeHelpers.addEmbeddingsToActivities(slimmedActsDf, embeddings)

In [15]:
resultsDf = pd.DataFrame()

In [16]:
for label in ["JOB_ID", "JOB_SCORES"]:
    resultsDf[label] = jobsDf[label]

In [17]:
resultsDf = resultsDf.merge(slimmedActsDf, how = "cross")

In [18]:
resultsDf = resultsDf[["JOB_ID", "ACTIVITY_ID", "JOB_SCORES", "ACTIVITY_SCORES"]]

In [19]:
def compareSentences(row):
    for sentence in resultsDf["JOB_SCORES"]:
        return spatial.distance.cosine(row["JOB_SCORES"][sentence], row["ACTIVITY_SCORES"])

In [20]:
def distCalc(row):
    distArray = []
    for entry in row["JOB_SCORES"] :
        dist = spatial.distance.cosine(entry, row["ACTIVITY_SCORES"])
        distArray.append(dist)
    return distArray
    

In [21]:
resultsDf["DISTANCES"] = resultsDf.apply(distCalc, axis =1)

In [23]:
def minDistanceFinder(row) : 
    minDistance = min(row["DISTANCES"])
    return minDistance

In [24]:
def minDistanceSentence(row, jobs) : 
    minDistanceIdx = np.argmin(row["DISTANCES"])
    df = jobs[jobs["JOB_ID"] == row["JOB_ID"]]
    if df.empty:
        return ""
    content = df.iloc[0]["JOB_CONTENT"]
    return content[minDistanceIdx]

In [25]:
resultsDf["LOWEST_DISTANCE"] = resultsDf.apply(minDistanceFinder, axis =1)
resultsDf["LOWEST_DISTANCE_SENTENCE"] = resultsDf.apply(minDistanceSentence, args=[jobsDf], axis =1)


In [26]:
resultsDf

,JOB_ID,ACTIVITY_ID,JOB_SCORES,ACTIVITY_SCORES,DISTANCES,LOWEST_DISTANCE,LOWEST_DISTANCE_SENTENCE
0,12220025,2,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.027285306, shape=(), dtype=float3...","[0.9204563945531845, 0.9612990096211433, 0.896...",0.670784,You want to gain experience in configuration ...
1,12220025,3,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.07172423, shape=(), dtype=float32...","[1.1060239672660828, 0.9814335498958826, 0.960...",0.730214,Further information and application We are at...
2,12220025,4,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.045720328, shape=(), dtype=float3...","[1.0675544813275337, 0.9523585364222527, 0.962...",0.807079,Haldi Design AG Schönried
3,12220025,5,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.05692438, shape=(), dtype=float32...","[1.0622159615159035, 0.968716274946928, 0.8875...",0.866480,Haldi Design AG Schönried
4,12220025,6,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.07220104, shape=(), dtype=float32...","[1.0261805169284344, 0.8907390013337135, 0.903...",0.873732,You want to gain experience in configuration ...
...,...,...,...,...,...,...,...
287995,14795850,290,"((tf.Tensor(0.018810535, shape=(), dtype=float...","(tf.Tensor(0.027627833, shape=(), dtype=float3...","[0.9140721634030342, 1.0835484191775322, 1.110...",0.754286,Exposure to manufacturing operations and or ...
287996,14795850,291,"((tf.Tensor(0.018810535, shape=(), dtype=float...","(tf.Tensor(-0.008161316, shape=(), dtype=float...","[0.8664177656173706, 1.0288245286792517, 1.128...",0.860321,Lead the adoption and maintenance of applica...
287997,14795850,292,"((tf.Tensor(0.018810535, shape=(), dtype=float...","(tf.Tensor(0.019920168, shape=(), dtype=float3...","[1.0274987239390612, 0.9213418960571289, 0.982...",0.784405,Drive the implementation of the required act...
287998,14795850,293,"((tf.Tensor(0.018810535, shape=(), dtype=float...","(tf.Tensor(0.0046779006, shape=(), dtype=float...","[0.7686764299869537, 0.9782356545329094, 1.098...",0.744638,I came for the job I stay for the culture LIALC


In [27]:
resultsDf["AVERAGE_DISTANCE_3SENT"] = resultsDf.apply(dataframeHelpers.averageNSentences, args=[3], axis=1)

In [28]:
row = resultsDf.loc[2]
row

JOB_ID                                                               12220025
ACTIVITY_ID                                                                 4
JOB_SCORES                  ((tf.Tensor(-0.04822776, shape=(), dtype=float...
ACTIVITY_SCORES             (tf.Tensor(0.045720328, shape=(), dtype=float3...
DISTANCES                   [1.0675544813275337, 0.9523585364222527, 0.962...
LOWEST_DISTANCE                                                      0.807079
LOWEST_DISTANCE_SENTENCE                            Haldi Design AG Schönried
AVERAGE_DISTANCE_3SENT                                               0.854717
Name: 2, dtype: object

In [29]:
slimmedActsDf[slimmedActsDf["ACTIVITY_ID"] == row["ACTIVITY_ID"]]

,ACTIVITY_CONTENT,ACTIVITY_ID,ACTIVITY_SCORES
2,Research geological features or processes.,4,"(tf.Tensor(0.045720328, shape=(), dtype=float3..."


In [30]:
jobMatch = resultsDf[resultsDf["JOB_ID"] == row["JOB_ID"]]
minEntries = jobMatch["LOWEST_DISTANCE"].argsort()[:10]

In [ ]:
#slimmedActsDf[slimmedActsDf["ACTIVITY_ID"].isin(bestMatch.loc[minEntries,:]["ACTIVITY_ID"])]["ACTIVITY_CONTENT"]

In [31]:
resultsDf

,JOB_ID,ACTIVITY_ID,JOB_SCORES,ACTIVITY_SCORES,DISTANCES,LOWEST_DISTANCE,LOWEST_DISTANCE_SENTENCE,AVERAGE_DISTANCE_3SENT
0,12220025,2,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.027285306, shape=(), dtype=float3...","[0.9204563945531845, 0.9612990096211433, 0.896...",0.670784,You want to gain experience in configuration ...,0.764877
1,12220025,3,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.07172423, shape=(), dtype=float32...","[1.1060239672660828, 0.9814335498958826, 0.960...",0.730214,Further information and application We are at...,0.841840
2,12220025,4,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.045720328, shape=(), dtype=float3...","[1.0675544813275337, 0.9523585364222527, 0.962...",0.807079,Haldi Design AG Schönried,0.854717
3,12220025,5,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.05692438, shape=(), dtype=float32...","[1.0622159615159035, 0.968716274946928, 0.8875...",0.866480,Haldi Design AG Schönried,0.889076
4,12220025,6,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.07220104, shape=(), dtype=float32...","[1.0261805169284344, 0.8907390013337135, 0.903...",0.873732,You want to gain experience in configuration ...,0.885543
...,...,...,...,...,...,...,...,...
287995,14795850,290,"((tf.Tensor(0.018810535, shape=(), dtype=float...","(tf.Tensor(0.027627833, shape=(), dtype=float3...","[0.9140721634030342, 1.0835484191775322, 1.110...",0.754286,Exposure to manufacturing operations and or ...,0.793880
287996,14795850,291,"((tf.Tensor(0.018810535, shape=(), dtype=float...","(tf.Tensor(-0.008161316, shape=(), dtype=float...","[0.8664177656173706, 1.0288245286792517, 1.128...",0.860321,Lead the adoption and maintenance of applica...,0.865023
287997,14795850,292,"((tf.Tensor(0.018810535, shape=(), dtype=float...","(tf.Tensor(0.019920168, shape=(), dtype=float3...","[1.0274987239390612, 0.9213418960571289, 0.982...",0.784405,Drive the implementation of the required act...,0.852274
287998,14795850,293,"((tf.Tensor(0.018810535, shape=(), dtype=float...","(tf.Tensor(0.0046779006, shape=(), dtype=float...","[0.7686764299869537, 0.9782356545329094, 1.098...",0.744638,I came for the job I stay for the culture LIALC,0.760556


In [32]:
resultsDf["JOB_SCORES"]

0         ((tf.Tensor(-0.04822776, shape=(), dtype=float...
1         ((tf.Tensor(-0.04822776, shape=(), dtype=float...
2         ((tf.Tensor(-0.04822776, shape=(), dtype=float...
3         ((tf.Tensor(-0.04822776, shape=(), dtype=float...
4         ((tf.Tensor(-0.04822776, shape=(), dtype=float...
                                ...                        
287995    ((tf.Tensor(0.018810535, shape=(), dtype=float...
287996    ((tf.Tensor(0.018810535, shape=(), dtype=float...
287997    ((tf.Tensor(0.018810535, shape=(), dtype=float...
287998    ((tf.Tensor(0.018810535, shape=(), dtype=float...
287999    ((tf.Tensor(0.018810535, shape=(), dtype=float...
Name: JOB_SCORES, Length: 288000, dtype: object

In [ ]:
#def activityMatcher(row) :
 #   for id in jobsDf["JOB_ID"] :
  #      bestOverallActivityMatch = np.argmin(resultsDf[row["JOB_ID"] == id]["LOWEST_DISTANCE"])
  #  return bestOverallActivityMatch 
    

In [33]:
column_names = ["JOB_ID", "ISCO", "JOB_CONTENT", "TOP_ACTIVITY_MATCHES", "TOP_ACTIVITY_IDS", "BEST_ACTIVITY_MATCH_3SENT", "BEST_ACTIVITY_ID_3SENT", "BEST_ACTIVITY_MATCH_1SENT", "BEST_ACTIVITY_ID_1SENT", "LOWEST_DISTANCE", ]

In [34]:
topResultsDf = pd.DataFrame(columns = column_names)

In [35]:
for label in ["JOB_ID", "ISCO", "JOB_CONTENT", "ORIGINAL_CONTENT"]:
    topResultsDf[label] = jobsDf[label]

In [36]:
topResultsDf["JOB_CONTENT"] = jobsDf.apply(lambda x: " ".join(x["JOB_CONTENT"]), axis=1)

In [39]:
resultsDf

,JOB_ID,ACTIVITY_ID,JOB_SCORES,ACTIVITY_SCORES,DISTANCES,LOWEST_DISTANCE,LOWEST_DISTANCE_SENTENCE,AVERAGE_DISTANCE_3SENT
0,12220025,2,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.027285306, shape=(), dtype=float3...","[0.9204563945531845, 0.9612990096211433, 0.896...",0.670784,You want to gain experience in configuration ...,0.764877
1,12220025,3,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.07172423, shape=(), dtype=float32...","[1.1060239672660828, 0.9814335498958826, 0.960...",0.730214,Further information and application We are at...,0.841840
2,12220025,4,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.045720328, shape=(), dtype=float3...","[1.0675544813275337, 0.9523585364222527, 0.962...",0.807079,Haldi Design AG Schönried,0.854717
3,12220025,5,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.05692438, shape=(), dtype=float32...","[1.0622159615159035, 0.968716274946928, 0.8875...",0.866480,Haldi Design AG Schönried,0.889076
4,12220025,6,"((tf.Tensor(-0.04822776, shape=(), dtype=float...","(tf.Tensor(0.07220104, shape=(), dtype=float32...","[1.0261805169284344, 0.8907390013337135, 0.903...",0.873732,You want to gain experience in configuration ...,0.885543
...,...,...,...,...,...,...,...,...
287995,14795850,290,"((tf.Tensor(0.018810535, shape=(), dtype=float...","(tf.Tensor(0.027627833, shape=(), dtype=float3...","[0.9140721634030342, 1.0835484191775322, 1.110...",0.754286,Exposure to manufacturing operations and or ...,0.793880
287996,14795850,291,"((tf.Tensor(0.018810535, shape=(), dtype=float...","(tf.Tensor(-0.008161316, shape=(), dtype=float...","[0.8664177656173706, 1.0288245286792517, 1.128...",0.860321,Lead the adoption and maintenance of applica...,0.865023
287997,14795850,292,"((tf.Tensor(0.018810535, shape=(), dtype=float...","(tf.Tensor(0.019920168, shape=(), dtype=float3...","[1.0274987239390612, 0.9213418960571289, 0.982...",0.784405,Drive the implementation of the required act...,0.852274
287998,14795850,293,"((tf.Tensor(0.018810535, shape=(), dtype=float...","(tf.Tensor(0.0046779006, shape=(), dtype=float...","[0.7686764299869537, 0.9782356545329094, 1.098...",0.744638,I came for the job I stay for the culture LIALC,0.760556


In [40]:
#added this
cutoff = resultsDf["AVERAGE_DISTANCE_3SENT"].mean() - resultsDf["AVERAGE_DISTANCE_3SENT"].std()
print("The cutoff is mean-std for all the average 3 sent. Value is", cutoff)

The cutoff is mean-std for all the average 3 sent. Value is 0.8491833441602706


In [41]:
#added this
topResultsDf["LOWEST_DISTANCE"] = resultsDf["LOWEST_DISTANCE"]
topResultsDf["AVERAGE_DISTANCE_3SENT"] = resultsDf["AVERAGE_DISTANCE_3SENT"]

In [ ]:
topResultsDf

In [48]:
topResultsDf["BEST_ACTIVITY_MATCH_1SENT"] = topResultsDf.apply(dataframeHelpers.bestMatchReturn, args=(resultsDf, slimmedActsDf, "ACTIVITY_ID", "LOWEST_DISTANCE", "ACTIVITY_CONTENT"), axis=1)
topResultsDf["BEST_ACTIVITY_MATCH_3SENT"] = topResultsDf.apply(dataframeHelpers.bestMatchReturn, args=(resultsDf, slimmedActsDf, "ACTIVITY_ID", "AVERAGE_DISTANCE_3SENT", "ACTIVITY_CONTENT"), axis=1)
topResultsDf["BEST_ACTIVITY_ID_1SENT"] = topResultsDf.apply(dataframeHelpers.bestMatchReturn, args=(resultsDf, slimmedActsDf, "ACTIVITY_ID", "LOWEST_DISTANCE", "ACTIVITY_ID"), axis=1)
topResultsDf["BEST_ACTIVITY_ID_3SENT"] = topResultsDf.apply(dataframeHelpers.bestMatchReturn, args=(resultsDf, slimmedActsDf, "ACTIVITY_ID", "AVERAGE_DISTANCE_3SENT", "ACTIVITY_ID"), axis=1)


In [50]:
topResultsDf["TOP_ACTIVITY_IDS"] = topResultsDf.apply(dataframeHelpers.bestNMatchReturn, args=(resultsDf, slimmedActsDf, "ACTIVITY_ID", "AVERAGE_DISTANCE_3SENT", "ACTIVITY_ID", 10), axis=1)
topResultsDf["TOP_ACTIVITY_MATCHES"] = topResultsDf.apply(dataframeHelpers.bestNMatchReturn, args=(resultsDf, slimmedActsDf, "ACTIVITY_ID", "AVERAGE_DISTANCE_3SENT", "ACTIVITY_CONTENT", 10), axis=1)
topResultsDf

,JOB_ID,ISCO,JOB_CONTENT,TOP_ACTIVITY_MATCHES,TOP_ACTIVITY_IDS,BEST_ACTIVITY_MATCH_3SENT,BEST_ACTIVITY_ID_3SENT,BEST_ACTIVITY_MATCH_1SENT,BEST_ACTIVITY_ID_1SENT,LOWEST_DISTANCE,ORIGINAL_CONTENT,AVERAGE_DISTANCE_3SENT
0,12220025,21420,We are a successful and independent engineerin...,[Manage engineering or construction activities...,"[257, 144, 112, 163, 286, 2, 155, 122, 111, 62]",Manage engineering or construction activities.,257,Manage engineering or construction activities.,257,0.670784,<p>Wir sind ein erfolgreiches und unabhängiges...,0.764877
1,10814567,21420,We are a prestigious and successful engineerin...,[Manage engineering or construction activities...,"[257, 286, 112, 141, 185, 282, 211, 196, 55, 119]",Manage engineering or construction activities.,257,Manage engineering or construction activities.,257,0.730214,Wir sind eine renommierte und erfolgreiche Ing...,0.841840
2,12401674,21420,Bill Weyermann Partner AG has been producing e...,[Manage engineering or construction activities...,"[257, 286, 112, 111, 155, 263, 134, 166, 185, ...",Manage engineering or construction activities.,257,Design civil structures or systems.,112,0.807079,Bill Weyermann Partner AG erbringt seit über\n...,0.854717
3,12473085,62100,Administration Communale de Valderuzforstartin...,[Design energy-efficient equipment or systems....,"[142, 244, 136, 174, 212, 156, 38, 128, 259, 258]",Design energy-efficient equipment or systems.,142,"Identifying the educational needs of others, ...",259,0.866480,Administration communale de Val-de-RuzForstwar...,0.889076
4,12905109,26220;24210,The Straumann Group is experiencing internatio...,[Develop systems or practices to mitigate or r...,"[113, 155, 93, 255, 145, 283, 95, 273, 233, 280]",Develop systems or practices to mitigate or re...,113,Develop systems or practices to mitigate or re...,113,0.873732,Le groupe Straumann connaît un succès internat...,0.885543
...,...,...,...,...,...,...,...,...,...,...,...,...
995,17126875,24210,Vallorbe SA Metallurgical factories are recogn...,[Monitor operational procedures in technical e...,"[43, 219, 143, 192, 139, 182, 193, 291, 21, 59]",Monitor operational procedures in technical en...,43,Monitor operational procedures in technical en...,43,0.986664,<p>Les Usines Métallurgiques de Vallorbe SA (U...,1.021114
996,17140922,43210;24210,Back to Search Results Apply for this item OME...,[ Monitoring and controlling resources and ove...,"[294, 144, 220, 264, 291, 145, 12, 155, 45, 14]",Monitoring and controlling resources and over...,294,Monitoring and controlling resources and over...,294,0.962983,<h2>Retour aux résultats de la recherche</h2>\...,0.978247
997,17140923,24210,Back to Search Results Apply for this item Ome...,[Keeping up-to-date technically and applying n...,"[144, 233, 276, 145, 14, 259, 220, 83, 294, 155]",Keeping up-to-date technically and applying ne...,144,Keeping up-to-date technically and applying ne...,144,1.032992,<h2>Retour aux résultats de la recherche</h2>\...,1.061759
998,17060287,24210,You are here HomePlayController Quality HF Qua...,[Consult with other professionals on green iss...,"[231, 287, 62, 43, 155, 29, 145, 163, 70, 212]",Consult with other professionals on green issues.,231,"Servicing, repairing, adjusting, and testing m...",198,0.976436,Vous êtes ici :AccueilEmploisContrôleur qualit...,0.993430


In [ ]:
annotatedJobs200 = pd.read_csv("Data/AnnotatedJobs200.csv", sep=";")

In [ ]:
topResultsDf["GREEN_IDENTITY"] = annotatedJobs200["GREEN_IDENTITY"].astype('bool')

In [ ]:
topResultsDf = topResultsDf.sort_values(axis=0, by="AVERAGE_DISTANCE_3SENT")

In [ ]:
topResultsDf

In [ ]:
#topResultsDf.to_csv("PrelimActsResults200V26.csv")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(topResultsDf["AVERAGE_DISTANCE_3SENT"], color = 'green', density=True, label="Activity with best match", alpha=0.5)
plt.hist(resultsDf["AVERAGE_DISTANCE_3SENT"], color = 'brown', density=True, label="All activities", alpha=0.5)
plt.xlabel("Cosine distance")
plt.ylabel("Normalized counts")
plt.legend()

In [ ]:
plt.hist(topResultsDf["AVERAGE_DISTANCE_3SENT"][topResultsDf["GREEN_IDENTITY"] == 1.0], label="Environmentally related jobs", color='green', density=True, alpha=0.5)
plt.hist(topResultsDf["AVERAGE_DISTANCE_3SENT"][topResultsDf["GREEN_IDENTITY"] == 0.], label="Non-environnmentally related jobs", color='brown', density=True, alpha=0.5)
plt.xlabel("Cosine distance")
plt.ylabel("Normalized counts")
plt.legend()

In [ ]:
topResultsDf[topResultsDf["AVERAGE_DISTANCE_3SENT"] < 0.85].to_csv("PrelimResultsActs200V26_GoodMatches.csv")

In [22]:
plt.hist(topResultsDf["AVERAGE_DISTANCE_3SENT"], density=True, label="Best match", color =  alpha=0.5)
plt.hist(resultsDf["AVERAGE_DISTANCE_3SENT"], density=True, label="all activities", alpha=0.5)
plt.axvline(np.mean(topResultsDf["AVERAGE_DISTANCE_3SENT"]), color='blue', linestyle='dashed')
plt.axvline(np.mean(resultsDf["AVERAGE_DISTANCE_3SENT"]), color='black', linestyle='dashed')
plt.legend()

SyntaxError: invalid syntax (<ipython-input-22-938d4e2a62bd>, line 1)

In [ ]:
resultsDf["AVERAGE_DISTANCE_3SENT"].mean()-resultsDf["AVERAGE_DISTANCE_3SENT"].std()